In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [14]:
ee.Initialize()

daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4') \
     .filter(ee.Filter.date('2019-01-01', '2019-12-31')) \
     .mean() ##Will mess up the toBands() function if mean is included - see below (daymet2) for this operation
##Note: there are 7 measurements per day (2555 per year except on leapyears) 

# daymet2 = ee.ImageCollection('NASA/ORNL/DAYMET_V4') \
#      .filter(ee.Filter.date('2000-01-01', '2000-12-31'))

roi = ee.Geometry.Polygon(
        [
            [
                [-81.1093, 31.6045],
                [-81.2000, 31.2060],
                [-81.5351, 31.2623],
                [-81.4280, 31.6349]
            ]
        ]
    )

dm_precip = daymet.select('prcp')
dm_maxtemp = daymet.select('tmax')
dm_mintemp = daymet.select('tmin')
dm_daylight = daymet.select('dayl')
dm_srad = daymet.select('srad')
dm_snow = daymet.select('swe')
dm_vapor = daymet.select('vp')

prcpVis = {
  'min': -40.0,
  'max': 30.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

daylVis = {
  'min': 0,
  'max': 50000.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

sradVis = {
  'min': 0,
  'max': 400.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

vpVis = {
  'min': 0,
  'max': 1500,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = geemap.Map(center=[40,-100], zoom=4)
Map.addLayer(dm_precip, prcpVis, "Precipitation")
Map.addLayer(dm_maxtemp, prcpVis, "Maxtemp")
Map.addLayer(dm_mintemp, prcpVis, "Mintemp")
Map.addLayer(dm_daylight, daylVis, "Daylight")
Map.addLayer(dm_srad, sradVis, "SRAD")
Map.addLayer(dm_snow, prcpVis, "Snowpack water?")
Map.addLayer(dm_vapor, vpVis, "Vaporpressure")

Map.addLayer(roi, name = "Region of interest")

Map

##For the year 2000, site near blackbeard creek entrance is missing daymet data! Check to see if that's the same for all years
##EDIT: it looks like it's going to be for all years

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

<IPython.core.display.Javascript object>

In [15]:
# daymet.getInfo()
# daymet2.getInfo()

In [4]:
##Convert imagecollection to bands/ee.image, from: 
##https://ecodata.nrel.colostate.edu/gdpe-gee-remote-sensing-lessons/module4-interoperability.html#daySmet-weather-data

# DaymetImage = daymet2.toBands()

##check this site out to try to average bands for analysis: 
##https://gis.stackexchange.com/questions/384421/temporally-reducing-image-bands-using-google-earth-engine

# Mean_dm = daymet \
#   .toBands() \
#   .filterBounds(geometry)

# print(DaymetImage, "DaymetImage")

In [5]:
##Exporting pixel values from points
coords_year = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Daymet/GA/Coords_year/NoFlower/coords_2019.csv'
##NoFlower folder is identical to Coords_year folder, with flowering status removed (confuses csv_to_ee function)
fc_coords = geemap.csv_to_ee(coords_year, latitude = "Latitude", longitude = "Longitude")

Map.addLayer(fc_coords, {}, "fc_coords")

##export data
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'daymet2019.csv')

# geemap.extract_values_to_points(fc_coords, daymet, out_csv, scale = 500) #Scale specified due to error.MAKE SURE it's right

##note: scale = 500 was chosen because 1000 yielded more NaNs. Same results as scale = 1000

##Deprecated:
# geemap.extract_values_to_points(fc_coords, DaymetImage, out_csv)
#out_shp = os.path.join(out_dir, 'points.shp') ##shapefile exporting

In [23]:
##Interpolating daymet data for missing site

##NOT SURE HOW THIS WILL WORK

samples = daymet.addBands(ee.Image.pixelLonLat()) \
  .sample({'region': roi, 'numPixels': 1500,
    'scale':1000, 'projection': 'EPSG:4326'})

def func_xkv(sample):
    lat = sample.get('latitude')
    lon = sample.get('longitude')
    prcp = sample.get('prcp')
    tmax = sample.get('tmax')
    tmin = sample.get('tmin')
    srad = sample.get('srad')
    vp = sample.get('vp')
    swe = sample.get('swe')
    dayl = sample.get('dayl')
    return ee.Feature(ee.Geometry.Point([lon, lat]), {'prcp': prcp}, {'tmax': tmax}, {'tmin': tmin}, {'srad': srad} \
                     , {'vp': vp}, {'swe': swe}, {'dayl': dayl}) \
  .map(func_xkv)

# Combine mean and standard deviation reducers for efficiency.
combinedReducer = ee.Reducer.mean().combine({
  'reducer2': ee.Reducer.stdDev(),
  'sharedInputs': True})

# Estimate global mean and standard deviation from the points.
stats = samples.reduceColumns({
  'reducer': combinedReducer,
  'selectors': ['prcp', 'tmax', 'tmin', 'srad', 'vp', 'swe', 'dayl']})

# Do the interpolation, valid to 70 kilometers.
interpolated = samples.inverseDistance({
  'range': 10,
  'propertyName': 'ch4',
  'mean': stats.get('mean'),
  'stdDev': stats.get('stdDev'),
  'gamma': 0.3})

# Define visualization arguments.
band_viz = {
  'min': 1800,
  'max': 1900,
  'palette': ['0D0887', '5B02A3', '9A179B', 'CB4678',
            'EB7852', 'FBB32F', 'F0F921']}

# Display to map.
Map.centerObject(aoi, 7)
Map.addLayer(ch4, band_viz, 'CH4')
Map.addLayer(interpolated, band_viz, 'CH4 Interpolated')
Map

EEException: Invalid GeoJSON geometry.

In [6]:
# DaymetImage.getInfo()

In [7]:
##Seasonal, rather than yearly, averages

dm_season = ee.ImageCollection('NASA/ORNL/DAYMET_V4') \
     .filter(ee.Filter.date('2000-03-01', '2000-10-31')) \
     .mean()

season_precip = dm_season.select('prcp')
Map.addLayer(season_precip, prcpVis, "Growing season prcp")

##Note: there are 7 measurements per day (2555 per year except on leapyears) 

##Exporting pixel values from points
coords_year = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Daymet/GA/Coords_year/NoFlower/coords_2000.csv'
##NoFlower folder is identical to Coords_year folder, with flowering status removed (confuses csv_to_ee function)
sn_coords = geemap.csv_to_ee(coords_year, latitude = "Latitude", longitude = "Longitude")

Map.addLayer(fc_coords, {}, "sn_coords")

##export data
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'dmseason2000.csv')

# geemap.extract_values_to_points(sn_coords, dm_season, out_csv, scale = 500) #Scale specified due to error.MAKE SURE it's right
